In [ ]:
from configparser import ConfigParser
import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np

parser = ConfigParser()
parser.read('./spotify_credentials.cfg')

SPOTIPY_CLIENT_ID = parser.get('spotify', 'SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = parser.get('spotify', 'SPOTIPY_CLIENT_SECRET')

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

In [ ]:
input = pd.read_csv('artists.csv', header = None)
artist_ids = []
for link in input[0]:
    
    artist_ids.append(link[32:54])

artist_ids

In [89]:
albums = []
album_track_ids = []
album_ids = []
artist_uri = 'spotify:artist:' + artist_ids[0]

# Add album ids to album_ids list
results = sp.artist_albums(artist_uri, album_type='album')
albums.extend(results['items'])
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])
for album in albums:
    album_ids.append(album['id'])

all_tracks = []
for id in album_ids:

    track_ids = []

    results = sp.album_tracks(id)
    tracks = results['items']
    all_tracks.extend(tracks)
    for track in tracks:
        track_ids.append(track['id'])

    album_track_ids.append(track_ids)


assert len(album_track_ids) == len(albums)

len(albums)

22

In [90]:
# albums = albums[0:2]
album_track_ids = album_track_ids[0:2]
album_track_ids

[['0dEIca2nhcxDUV8C5QkPYb',
  '3ctALmweZBapfBdFiIVpji',
  '0oks4FnzhNp5QPTZtoet7c',
  '7Bxv0WL7UC6WwQpk9TzdMJ',
  '2cGxRwrMyEAp8dEbuZaVv6',
  '5CMjjywI0eZMixPeqNd75R',
  '7oaEjLP2dTJLJsITbAxTOz',
  '69kOkLUCkxIZYexIgSG8rq',
  '0k1xMUwn9sb7bZiqdT9ygx',
  '79koEJRtKOOGJ0VSAF3FMk',
  '0IedgQjjJ8Ad4B3UDQ5Lyn',
  '36c4JohayB9qd64eidQMBi',
  '2KHRENHQzTIQ001nlP9Gdc'],
 ['4l2bO887Yauqos4LwkCwBm',
  '3K6HXNtxo1A2eCthCzrwaS',
  '2fJp2bDmhG62vlmUpuWjxb',
  '5pZHN9b6KrXlUC5Dt3FA0O',
  '4I2apNr2XWqSo2UdEFwUsv',
  '6TOFKxiVogOtLKEBGMIFX7',
  '40DD7ZKO1urSddN6HhMfuh',
  '4TgBZlDIwm4kagFvqnw08R']]

In [40]:
# sp.audio_features('2RpXUwCytDFqZSHl6kIquB')
rty = sp.audio_analysis('2RpXUwCytDFqZSHl6kIquB')['segments'][0]['pitches']

In [ ]:
# columns_to_remove = ['analysis_url', 'type', 'uri', 'track_href']
headings = ['start',
'duration',
'confidence',
'loudness',
'tempo',
'tempo_confidence',
'key',
'key_confidence',
'mode',
'mode_confidence',
'time_signature',
'time_signature_confidence']
data = pd.DataFrame(columns=headings)

# Create DataFrames of features from track ids
i = 0
j = 0
for album in album_track_ids:
    
    length = len(album)
    
    for track in album:
        track_sections = sp.audio_analysis(track)['sections']
        for results in track_sections:
            if results: 
                features = results
            features_matrix = pd.DataFrame.from_records(features, index=[0])
            
            # Remove unneeded columns
            # features_matrix.drop(columns = columns_to_remove, axis = 1, inplace = True)
            # print(features_matrix)

            # Add track title
            track_title = all_tracks[j]['name']
            features_matrix['track_title'] = track_title
            # Add album title
            album_title = albums[i]['name']
            features_matrix['album_title'] = album_title
            # Add artist name
            name = albums[i]['artists'][0]['name']
            features_matrix['album_artist'] = name

            # Add track number and total tracks
            features_matrix['track_number'] = all_tracks[j]['track_number']
            features_matrix['total_tracks'] = length

            # Add album as a row to the data df
            # print(features_matrix,'\n')
            # print(data)
            data = data.append(features_matrix)

        j += 1

    i += 1

# Remove duplicates
data.drop_duplicates(subset=['track_number', 'album_title', 'album_artist'])
data

In [ ]:
data['order'] = data['track_number']/data['total_tracks']

data.to_csv('./data/sections.csv')
data

In [91]:
# columns_to_remove = ['analysis_url', 'type', 'uri', 'track_href']
headings = ['start',
            'duration',
            'confidence',
            'loudness_start',
            'loudness_max_time',
            'loudness_max',
            'loudness_end']#, 'Pitches', 'Timbres']
data = pd.DataFrame(columns=headings)

# Create DataFrames of features from track ids
i = 0
j = 0
k = 0
pitches = []
timbres = []
for album in album_track_ids:
    for track in album:
        track_segments = sp.audio_analysis(track)['segments']
        for result in track_segments:
            pitches.append(result.get('pitches'))
            del result['pitches']
            timbres.append(result.get('timbre'))
            del result['timbre']
            k += 1

        for results in track_segments:
            if results: 
                features = results
            features_matrix = pd.DataFrame.from_records(features, index=[0])
            
            # Remove unneeded columns
            # # features_matrix.drop(columns = columns_to_remove, axis = 1, inplace = True)
            # # print(features_matrix)

            # Add track title
            track_title = all_tracks[j]['name']
            features_matrix['track_title'] = track_title
            # Add album title
            album_title = albums[i]['name']
            features_matrix['album_title'] = album_title
            # Add artist name
            name = albums[i]['artists'][0]['name']
            features_matrix['album_artist'] = name

            # Add track number and total tracks
            features_matrix['track_number'] = all_tracks[j]['track_number']
            features_matrix['total_tracks'] = len(album)

            # Add album as a row to the data df
            data = pd.concat([data, features_matrix])

        j += 1

    i += 1

data['pitches'] = pitches
data['timbre'] = timbres

# Remove duplicates
# data.drop_duplicates(subset=['track_number', 'album_title', 'album_artist'])

data

,start,duration,confidence,loudness_start,loudness_max_time,loudness_max,loudness_end,track_title,album_title,album_artist,track_number,total_tracks,pitches,timbre
0,0.00000,1.01583,0.000,-60.000,0.00000,-60.000,0.0,Give Life Back to Music,Random Access Memories,Daft Punk,1.0,13.0,"[0.202, 0.246, 0.215, 0.198, 0.407, 1.0, 0.246...","[0.0, 171.13, 9.469, -28.48, 57.491, -50.067, ..."
0,1.01583,0.31596,1.000,-60.000,0.07260,-9.859,0.0,Give Life Back to Music,Random Access Memories,Daft Punk,1.0,13.0,"[0.169, 0.14, 0.057, 0.42, 1.0, 0.139, 0.033, ...","[35.458, 55.906, 69.742, -166.536, -3.252, 103..."
0,1.33179,0.20694,0.630,-16.198,0.00584,-9.356,0.0,Give Life Back to Music,Random Access Memories,Daft Punk,1.0,13.0,"[0.559, 0.518, 0.394, 0.09, 0.082, 0.069, 0.30...","[47.965, 57.123, -5.636, -1.102, -15.098, -44...."
0,1.53873,0.65542,0.721,-14.680,0.04780,-6.658,0.0,Give Life Back to Music,Random Access Memories,Daft Punk,1.0,13.0,"[0.156, 0.123, 0.097, 0.064, 0.292, 0.094, 0.0...","[49.72, 76.665, 1.289, -54.631, -18.938, 1.954..."
0,2.19415,0.13379,0.260,-11.337,0.05143,-8.078,0.0,Give Life Back to Music,Random Access Memories,Daft Punk,1.0,13.0,"[0.222, 0.188, 0.054, 0.109, 0.452, 0.159, 0.4...","[50.446, 81.393, 18.975, -21.829, -16.591, -18..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,446.85956,0.20059,1.000,-42.011,0.05542,-23.621,0.0,"Episode 8, Paul Williams",Daft Punk | Random Access Memories | The Colla...,Daft Punk,8.0,8.0,"[0.377, 1.0, 0.55, 0.527, 0.407, 0.295, 0.557,...","[29.922, -110.172, -13.061, 10.591, 11.36, 48...."
0,447.06015,0.13973,0.540,-38.548,0.03478,-32.115,0.0,"Episode 8, Paul Williams",Daft Punk | Random Access Memories | The Colla...,Daft Punk,8.0,8.0,"[0.665, 0.61, 0.481, 0.464, 0.84, 1.0, 0.25, 0...","[25.581, -94.848, 34.255, 8.047, -5.453, -18.7..."
0,447.19986,0.10921,0.088,-37.068,0.01982,-33.604,0.0,"Episode 8, Paul Williams",Daft Punk | Random Access Memories | The Colla...,Daft Punk,8.0,8.0,"[0.481, 1.0, 0.477, 0.477, 0.381, 0.383, 0.202...","[21.545, 201.24, -134.528, 7.301, -12.098, -37..."
0,447.30908,0.21193,1.000,-47.609,0.06644,-25.926,0.0,"Episode 8, Paul Williams",Daft Punk | Random Access Memories | The Colla...,Daft Punk,8.0,8.0,"[0.459, 0.923, 0.801, 0.512, 0.901, 1.0, 0.762...","[26.2, -55.72, 29.116, 52.232, 59.562, 77.648,..."


In [92]:
data['order'] = data['track_number']/data['total_tracks']

data.to_csv('./data/segments.csv')
data

,start,duration,confidence,loudness_start,loudness_max_time,loudness_max,loudness_end,track_title,album_title,album_artist,track_number,total_tracks,pitches,timbre,order
0,0.00000,1.01583,0.000,-60.000,0.00000,-60.000,0.0,Give Life Back to Music,Random Access Memories,Daft Punk,1.0,13.0,"[0.202, 0.246, 0.215, 0.198, 0.407, 1.0, 0.246...","[0.0, 171.13, 9.469, -28.48, 57.491, -50.067, ...",0.076923
0,1.01583,0.31596,1.000,-60.000,0.07260,-9.859,0.0,Give Life Back to Music,Random Access Memories,Daft Punk,1.0,13.0,"[0.169, 0.14, 0.057, 0.42, 1.0, 0.139, 0.033, ...","[35.458, 55.906, 69.742, -166.536, -3.252, 103...",0.076923
0,1.33179,0.20694,0.630,-16.198,0.00584,-9.356,0.0,Give Life Back to Music,Random Access Memories,Daft Punk,1.0,13.0,"[0.559, 0.518, 0.394, 0.09, 0.082, 0.069, 0.30...","[47.965, 57.123, -5.636, -1.102, -15.098, -44....",0.076923
0,1.53873,0.65542,0.721,-14.680,0.04780,-6.658,0.0,Give Life Back to Music,Random Access Memories,Daft Punk,1.0,13.0,"[0.156, 0.123, 0.097, 0.064, 0.292, 0.094, 0.0...","[49.72, 76.665, 1.289, -54.631, -18.938, 1.954...",0.076923
0,2.19415,0.13379,0.260,-11.337,0.05143,-8.078,0.0,Give Life Back to Music,Random Access Memories,Daft Punk,1.0,13.0,"[0.222, 0.188, 0.054, 0.109, 0.452, 0.159, 0.4...","[50.446, 81.393, 18.975, -21.829, -16.591, -18...",0.076923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,446.85956,0.20059,1.000,-42.011,0.05542,-23.621,0.0,"Episode 8, Paul Williams",Daft Punk | Random Access Memories | The Colla...,Daft Punk,8.0,8.0,"[0.377, 1.0, 0.55, 0.527, 0.407, 0.295, 0.557,...","[29.922, -110.172, -13.061, 10.591, 11.36, 48....",1.000000
0,447.06015,0.13973,0.540,-38.548,0.03478,-32.115,0.0,"Episode 8, Paul Williams",Daft Punk | Random Access Memories | The Colla...,Daft Punk,8.0,8.0,"[0.665, 0.61, 0.481, 0.464, 0.84, 1.0, 0.25, 0...","[25.581, -94.848, 34.255, 8.047, -5.453, -18.7...",1.000000
0,447.19986,0.10921,0.088,-37.068,0.01982,-33.604,0.0,"Episode 8, Paul Williams",Daft Punk | Random Access Memories | The Colla...,Daft Punk,8.0,8.0,"[0.481, 1.0, 0.477, 0.477, 0.381, 0.383, 0.202...","[21.545, 201.24, -134.528, 7.301, -12.098, -37...",1.000000
0,447.30908,0.21193,1.000,-47.609,0.06644,-25.926,0.0,"Episode 8, Paul Williams",Daft Punk | Random Access Memories | The Colla...,Daft Punk,8.0,8.0,"[0.459, 0.923, 0.801, 0.512, 0.901, 1.0, 0.762...","[26.2, -55.72, 29.116, 52.232, 59.562, 77.648,...",1.000000


In [30]:
# data = np.zeros((len(all_tracks*1500),9))
data = np.zeros((1,10))
titles = ['Start','Duration', 'Confidence', 'Loudness_Start',
            'Loudness_Max_Time', 'Loudness_Max', 'Loudness_Max',
            'Loudness_End', 'Pitches', 'Timbres']
result = sp.audio_analysis('2RpXUwCytDFqZSHl6kIquB')['segments']
data[0] = result[0].values()

temp = result[0].values()
temp

TypeError: float() argument must be a string or a number, not 'dict_values'